Training BERT Language Model From Scratch On TPUs

https://www.youtube.com/watch?v=s-3zts7FTDA&t=10s

https://www.kaggle.com/abhishek/training-language-models-on-tpus-from-scratch
#### In this kernel, I will show you how to train language models, such as BERT, from scratch on TPUs!

#### If you like this kernel, consider upvoting it and the associated datasets:

- https://www.kaggle.com/abhishek/bert-master
- https://www.kaggle.com/abhishek/hindi-oscar-corpus
- https://www.kaggle.com/abhishek/bert-base-uncased

In [1]:
! pip install -U tokenizers

     |████████████████████████████████| 3.7 MB 3.4 MB/s 
ERROR: transformers 2.4.1 has requirement tokenizers==0.0.11, but you'll have tokenizers 0.4.2 which is incompatible.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.0.11
    Uninstalling tokenizers-0.0.11:
      Successfully uninstalled tokenizers-0.0.11


In [2]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 27 kB/s 
     |████████████████████████████████| 3.8 MB 35.1 MB/s 
     |████████████████████████████████| 503 kB 47.7 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.1.0
    Uninstalling tensorflow-2.1.0:
      Successfully uninstalled tensorflow-2.1.0


In [3]:
import tokenizers

In [4]:
bwpt = tokenizers.BertWordPieceTokenizer(
    vocab_file=None,
    add_special_tokens=True,
    unk_token='[UNK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True,
    lowercase=True,
    wordpieces_prefix='##'
)

In [5]:
bwpt.train(
    files=["../input/hindi-oscar-corpus/hi_dedup_1000.txt"],
    vocab_size=30000,
    min_frequency=3,
    limit_alphabet=1000,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]']
)

In [6]:
bwpt.save("/kaggle/working/", "hindi")

['/kaggle/working/hindi-vocab.txt']

In [7]:
cd ../input/bertsrc/

/kaggle/input/bertsrc


In [8]:
!ls

CONTRIBUTING.md		    optimization_test.py
LICENSE			    predicting_movie_reviews_with_bert_on_tf_hub.ipynb
README.md		    requirements.txt
__init__.py		    run_classifier.py
create_pretraining_data.py  run_classifier_with_tfhub.py
extract_features.py	    run_pretraining.py
modeling.py		    run_squad.py
modeling_test.py	    sample_text.txt
multilingual.md		    tokenization.py
optimization.py		    tokenization_test.py


In [9]:
!python create_pretraining_data.py \
    --input_file=/kaggle/input/hindi-oscar-corpus/hi_dedup_1000.txt \
    --output_file=/kaggle/working/tf_examples.tfrecord \
    --vocab_file=/kaggle/working/hindi-vocab.txt \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=42 \
    --dupe_factor=5

W0215 07:16:52.058788 140411286930816 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0215 07:16:52.059109 140411286930816 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0215 07:16:52.059427 140411286930816 module_wrapper.py:139] From /kaggle/input/bertsrc/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0215 07:16:52.097698 140411286930816 module_wrapper.py:139] From create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0215 07:16:52.100157 140411286930816 module_wrapper.py:139] From create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I0215 07:16:52.100515 140411286930816 create_pretraining_data.py:4

In [10]:
!python run_pretraining.py \
    --input_file=gs://tf-lang-model/*.tfrecord \
    --output_dir=gs://tf-lang-model/model/ \
    --do_train=True \
    --do_eval=True \
    --bert_config_file=/kaggle/input/bert-base-uncased/config.json \
    --train_batch_size=32 \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --num_train_steps=20 \
    --num_warmup_steps=10 \
    --learning_rate=2e-5 \
    --use_tpu=True \
    --tpu_name=$TPU_NAME

W0215 07:17:07.396560 140162862937472 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0215 07:17:07.396826 140162862937472 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0215 07:17:07.397039 140162862937472 module_wrapper.py:139] From /kaggle/input/bertsrc/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0215 07:17:07.400176 140162862937472 module_wrapper.py:139] From run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

2020-02-15 07:17:19.469474: W tensorflow/core/platform/cloud/google_auth_provider.cc:178] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file."